In [1]:
import requests
import pandas as pd

from time import sleep

C:\Users\micha\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\micha\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\micha\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\micha\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# get active NFL players
def get_athletes_espn_data():
    """
        Get espn_id, player name, position, and team of every active player.
        https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/1/roster
            roster for Arizona Cards. 
    """
    athletes = {}

    for i in range(1,35): # iter through team ids (1-30, 33, 34)
        if i not in [31, 32]: # slow down
            sleep(.25)
            # get team roster
            url = f"https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{i}/roster"
            r = requests.get(url)
            data = r.json()

            # get athlete ids
            for athlete in data['athletes'][0]['items']: # offense
                athletes[athlete['id']] = {}
                athletes[athlete['id']]['displayName'] = athlete['displayName']
                athletes[athlete['id']]['position'] = athlete['position']['name']
                athletes[athlete['id']]['team'] = data['team']['displayName']
    return athletes

# get stats and fantasy points
def get_fantasy_stats(athlete, athlete_id, athletes, relevant_stats):
    """
        athlete dictionary from api.espn
            get stats that pertain to fantasy points
    """
    fantasy_stats = {}
    fantasy_stats['name'] = athletes[athlete_id]['displayName']
    for stat_category in athlete['splits']['categories']: # iter through stat types
        category = stat_category['name']

        for stat in stat_category['stats']:
            stat_name, stat_value = stat['name'], stat['displayValue'] 

            if (category in relevant_stats.keys()) and (stat_name in relevant_stats[category]):
                fantasy_stats[f"{category}_{stat['name']}"] = float(stat['displayValue'].replace(',',''))

    return fantasy_stats

def get_fantasy_points(fantasy_stats):
    """
    # Passing Yards: 1 point per 25 yards
    # Passing Touchdowns: 4 points
    # Passing Interceptions: -2 points

    # Rushing Yards: 1 point per 10 yards
    # Rushing Touchdowns: 6 points

    # Receptions: 1 points (only if using PPR scoring)
    # Receiving Yards: 1 point per 10 yards
    # Receiving Touchdowns: 6 points

    # 2-Point Conversions: 2 points
    # Fumbles Lost: -2 points
    # Fumble Recovered for a Touchdown: 6 points
    """
    
    points = 0      
    for stat in fantasy_stats:

        # passing
        if stat == 'passing_passingYards':
            # 1 point per 25 yards
            points += float(fantasy_stats[stat]) / 25 
        if stat == 'passing_passingTouchdowns':
            # 4 points per touchdown
            points += float(fantasy_stats[stat]) * 4  
        if stat == 'passing_interceptions':
            # -2 points per interception
            points -= float(fantasy_stats[stat]) * 2  
            
        # rushing
        if stat == 'rushing_rushingYards':
            # 1 point per 10 yards
            points += float(fantasy_stats[stat]) / 10
        if stat == 'rushing_rushingTouchdowns':
            # 6 point per touchdown
            points += float(fantasy_stats[stat]) * 6
            
        # receiving
        if stat == 'receiving_receivingYards':
            # 1 point per 10 yards
            points += float(fantasy_stats[stat]) / 10
        if stat == 'receiving_receivingTouchdowns':
            # 6 point per touchdown
            points += float(fantasy_stats[stat]) * 6

        # other
        if stat == 'scoring_totalTwoPointConvs':
            # 2 points per 2-Point Conversions 
            points += float(fantasy_stats[stat]) * 2
        if stat == 'general_fumblesLost':
            # -2 points per Fumbles Lost 
            points -= float(fantasy_stats[stat]) * 2
        if stat == 'general_fumblesTouchdowns':
            # 6 points per Fumble Recovered for Touchdown
            points += float(fantasy_stats[stat]) * 6

    return points
                              
def calc_athlete_fantasy_points(athletes):
    """
        per player get espn stast, calculate fantasy points
    """

    # stay displayNames to look for
    relevant_stat_names = {
        'passing': [
            'passingYards', 'passingTouchdowns', 'interceptions'
        ],
        'rushing': [
            'rushingYards', 'rushingTouchdowns'
        ],
        'receiving':[
            'receptions', 'receivingYards', 'receivingTouchdowns' 
        ],
        'general': [
            'fumblesLost', 'fumblesTouchdowns', 'gamesPlayed'
        ],
        'scoring': [
            'totalTwoPointConvs'
        ]
    }
    
    # create df 
    cols = [
        'name', 'team', 'position', 'year', 'fantasy_points', 
        'passing_passingYards','passing_passingTouchdowns', 'passing_interceptions', 
        'rushing_rushingYards', 'rushing_rushingTouchdowns',
        'receiving_receptions', 'receiving_receivingYards', 'receiving_receivingTouchdowns', 
        'general_fumblesLost', 'general_fumblesTouchdowns', 
        'scoring_totalTwoPointConvs'
    ]
    df = pd.DataFrame({}, columns=cols)
    
    year = "2023" # get fantasy stats and points for year
    for athlete_id in athletes:
        sleep(.25)
        # endpoint of athlete season stats
        url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{year}/types/2/athletes/{athlete_id}/statistics/"
        r_athlete = requests.get(url)
        data_athlete = r_athlete.json()

        if 'error' in data_athlete.keys():
    #         print(f"no stats for {athletes[athlete_id]['displayName']} in {year}")
            pass
        else: # we have data 
            fantasy_stats = get_fantasy_stats(athlete=data_athlete, 
                                    athlete_id=athlete_id, 
                                    athletes=athletes,
                                    relevant_stats=relevant_stat_names)
            fantasy_stats['team'] = athletes[athlete_id]['team']
            fantasy_stats['position'] = athletes[athlete_id]['position']
            fantasy_stats['fantasy_points'] = get_fantasy_points(fantasy_stats)  
            fantasy_stats['year'] = year    

            df.loc[len(df)] = fantasy_stats
    return df

In [3]:
# remove players from draft board
def take_off_the_board(df, athlete_name):
    df_updated = df
    
    if isinstance(athlete_name, list):
        df_updated = df.drop(df[df['name'].isin(names)].index)
    if isinstance(athlete_name, str):
        df_updated = df.drop(df[df['name'] == athlete_name].index)

    return df_updated

In [4]:
# determine who to draft next 
def highest_available(df,
        position='',
    ):

    if position:
        # slice df, sort by points, reset index to get top player
        draft_df = df[df['position']==position].sort_values('fantasy_points', ascending=False).reset_index().loc[0]
        return f"{draft_df['name']} ({draft_df['position']}), {draft_df['team']}"
    else:
        draft_df = df.sort_values('fantasy_points', ascending=False).reset_index().loc[0]
        return f"{draft_df['name']} ({draft_df['position']}), {draft_df['team']}"
    return

# run 

In [5]:
# get all active athletes
my_athletes = get_athletes_espn_data()

In [6]:
# calculate fantasy points
my_df = calc_athlete_fantasy_points(athletes=my_athletes)

In [7]:
my_df.sort_values('fantasy_points', ascending=False).head(10)

,name,team,position,year,fantasy_points,passing_passingYards,passing_passingTouchdowns,passing_interceptions,rushing_rushingYards,rushing_rushingTouchdowns,receiving_receptions,receiving_receivingYards,receiving_receivingTouchdowns,general_fumblesLost,general_fumblesTouchdowns,scoring_totalTwoPointConvs
581,Jalen Hurts,Philadelphia Eagles,Quarterback,2023,282.80,2995.0,19.0,10.0,430.0,12.0,0.0,0.0,0.0,4.0,0.0,0.0
28,Josh Allen,Buffalo Bills,Quarterback,2023,280.76,3214.0,24.0,13.0,342.0,9.0,0.0,0.0,0.0,3.0,0.0,0.0
154,Dak Prescott,Dallas Cowboys,Quarterback,2023,250.76,3234.0,26.0,6.0,174.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
702,Christian McCaffrey,San Francisco 49ers,Running Back,2023,244.10,0.0,0.0,0.0,1032.0,12.0,51.0,429.0,5.0,2.0,0.0,0.0
908,C.J. Stroud,Houston Texans,Quarterback,2023,237.90,3540.0,20.0,5.0,143.0,3.0,1.0,0.0,0.0,4.0,0.0,1.0
782,Sam Howell,Washington Commanders,Quarterback,2023,234.94,3466.0,18.0,14.0,243.0,5.0,0.0,0.0,0.0,2.0,0.0,1.0
667,Justin Herbert,Los Angeles Chargers,Quarterback,2023,229.32,3038.0,20.0,6.0,228.0,3.0,1.0,10.0,0.0,1.0,0.0,0.0
708,Brock Purdy,San Francisco 49ers,Quarterback,2023,228.50,3185.0,23.0,6.0,131.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
318,Patrick Mahomes,Kansas City Chiefs,Quarterback,2023,222.18,3127.0,22.0,10.0,331.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
865,Lamar Jackson,Baltimore Ravens,Quarterback,2023,222.12,2618.0,13.0,5.0,574.0,5.0,0.0,0.0,0.0,6.0,0.0,0.0


In [ ]:
# remove players that have been drafted (by name or by list of names)
# take_off_the_board(df=my, athlete_name='Josh Allen')
# take_off_the_board(df=my, athlete_name=['Josh Allen', 'Patrick Mahomes'])

In [8]:
# determine who to draft next (by highest points last season)
# highest_available(df=my_df, )# overall highest points
highest_available(df=my_df, position = 'Running Back') # by position

'Christian McCaffrey (Running Back), San Francisco 49ers'